In [3]:
import keras
import numpy as np
import numpy
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

In [35]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Dropout
from keras.optimizers import Adam

In [6]:
base_dir = '../data/'

In [7]:
train = pd.read_csv(base_dir+'train_1.csv', nrows=100).fillna(0)

In [8]:
len(train.columns)

551

In [9]:
train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36,10.0


In [109]:
scaler = MinMaxScaler(feature_range=(0, 1))
sequences = [row.values[1:] for _,row in train.iterrows()]

In [117]:
sequences = np.transpose(scaler.fit_transform(np.transpose(sequences)))

/Users/aliteeney/miniconda3/envs/explore/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [126]:
time_steps = 60

In [127]:
train_len = len(sequences[0])-time_steps ; train_len

490

In [128]:
def get_data_sequence(sequence, train_len):
    sequences = []
    next_elements = []
    for i in range(len(sequence)-train_len+1):
        sequences.append(sequence[i:i+train_len])
        next_elements.append(sequence[i+1:i+1+train_len])
    return np.stack(sequences[:-1]), np.stack(next_elements[:-1])

In [129]:
def get_data(sequences, train_len):
    return_sequences = []
    next_elements = []
    for s in sequences:
        ss, nes = get_data_sequence(s, train_len)
        return_sequences.append(ss)
        next_elements.append(nes)
    return_sequences = np.expand_dims(np.concatenate(return_sequences), axis=-1)
    next_elements = np.expand_dims(np.concatenate(next_elements), axis=-1)
    return return_sequences, next_elements

In [130]:
train_sequences, next_elements = get_data(sequences, train_len)

In [131]:
train_sequences[0][0]

array([ 0.03080082])

In [132]:
model=Sequential([
        LSTM(256, input_shape=(train_len, 1), return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(256, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(1))
    ])    

/Users/aliteeney/miniconda3/envs/explore/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(256, input_shape=(490, 1), return_sequences=True, dropout=0.2, recurrent_dropout=0.2, implementation=2)`
  app.launch_new_instance()
/Users/aliteeney/miniconda3/envs/explore/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, implementation=2)`


In [133]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 490, 256)          264192    
_________________________________________________________________
dropout_17 (Dropout)         (None, 490, 256)          0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 490, 256)          525312    
_________________________________________________________________
dropout_18 (Dropout)         (None, 490, 256)          0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 490, 1)            257       
Total params: 789,761
Trainable params: 789,761
Non-trainable params: 0
_________________________________________________________________


In [136]:
model.compile(loss='mean_squared_error', optimizer=Adam())

In [ ]:
model.fit(train_sequences, next_elements, batch_size=64, epochs=1)

Epoch 1/1
3520/6000 [================>.............] - ETA: 452s - loss: 0.0101